### 2.1 keras中的RNN
#### 一. SimpleRNN in keras
1. SimpleRNN的输入  
 SimpleRNN同时处理多条时序数据, 所以输入SimpleRNN的数据是3D张量 ( `batch_size`, `timestamps`, `input_features` )  
 如下例子中, Embedding层接上SimpleRNN层, 因为Embedding输出3D张量(sentences,word,embedding vector), 对应成SimpleRNN的3D输入 (batch_size,timestamps,input_features)
2. SimpleRNN的输出  
 keras中所有的RNN层都有两种模式, 通过参数return_sequences=True指定每个时间步都有输出  
  1. 时序数据中, 每个时间步都有输出. 进而形成3D张量 ( `batch_size`,`timestamps`,`output_features` )  
   如果想堆叠多个SimpleRNN, 则需要指定return_sequences=True
  2. 每个输入序列, 只在最后产生一个输出. 形成2D张量 ( `batch_size`,`output_features` )
3. 现在, 我们使用SimpleRNN来训练`imdb`数据

In [1]:
######################  准备imdb数据  ###################
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000
maxlen=500
batch_size=32

print 'loading data ... ... '
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=10000)
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=1000)
print 'finish ! '

/devkit/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


loading data ... ... 
finish ! 


In [2]:
print input_train[0]
print 'input_train shape: ', input_train.shape
print 'input_test shape: ', input_test.shape

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
input_train shape:  (25000,)
input_test shape:  (25000,)


In [3]:
# 取500个时间步长度 (截取语句500前个单词)
print 'Pad sequences (samples * time)'
# padding_sequence默认在句首填充,句首裁剪padding='pre', truncating='pre'
input_train = sequence.pad_sequences(input_train,maxlen=500)
input_test = sequence.pad_sequences(input_test,maxlen=500)
print 'input_train shape: ', input_train.shape
print 'input_test shape: ', input_test.shape
print 'input_train[0]:\n', input_train[0]  # 每个单词的编号

Pad sequences (samples * time)
input_train shape:  (25000, 500)
input_test shape:  (25000, 500)
input_train[0]:
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 

In [13]:
#####################  创建网络  ###################
from keras.models import Sequential
from keras.layers import Dense,Embedding,SimpleRNN

model = Sequential()
model.add(Embedding(10000,32))
# units : RNN的单元数，也是输出维度
model.add(SimpleRNN(32,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [14]:
#################   训练网络   #################
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(input_train,y_train,epochs=1,batch_size=32,validation_split=0.2)
print history.history['acc']

Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 87s 4ms/step - loss: 0.4851 - acc: 0.7706 - val_loss: 0.4169 - val_acc: 0.8238
[0.77065]


In [10]:
print history.history.keys()
0.5036

['acc', 'loss', 'val_acc', 'val_loss']


#### 二. LSTM in keras
1. 用LSTM解决imdb问题  
2. LSTM在imdb上的表现解释 :  
  1. 如上, 我们使用LSTM处理imdb数据集, 准确率达到89%. 比simplernn好很多, 这是因为LSTM可以处理长倚赖的梯度消失问题.    
  2. 同时, 也比直接用onehot编码后(向量中元素值为0/1)使用全连接网络训练的结果好.虽然全连接时考虑了语句中的所有单词, 而这里我们只是将每个句子拆成500个时间步(timestep).  
  3. 但是这个表现并不具有开创性, 原因可能是:   
    1. 没有调节用于LSTM的word embedding  
    2. 缺少正则化  
    3. 最主要的原因是 : 长倚赖序列模型, 并不最适合解决情感分析问题. 这类问题主要依赖于单次出现的频率. 而这个正是全连接网络所关注的.  
     LSTM真正的用处在于'问答'和'机器翻译'
3. LSTM类中的参数units :   
 1. units参数表示RNN单元中的隐藏节点个数.  隐藏节点就是下图中产生$a^{<t>}$的单元(由${ f }^{ <t> },{ i }^{ <t> },{ \widetilde { c }  }^{ <t> },{ o }^{ <t> }$组成). 因为每个隐藏节点会输出一个$a^{<t>}$, 所以隐藏节点个数就是LSTM单元的输出个数  
 2. 明确了LSTM单元的输出, 就得知, LSTM单元的输入维度为 : LSTM单元的units个数+每个时间步上输入向量的维度
 2. 相反, 时间步个数(LSTM单元个数)由输入sentence的单词个数决定  
<img src='../../img/lstm.png' height='80%' width='80%'>

In [15]:
from keras.layers import LSTM,Dense
model = Sequential()
model.add(Embedding(10000,32))
model.add(LSTM(32))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(input_train,y_train,epochs=1,batch_size=21,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 615s 31ms/step - loss: 0.4230 - acc: 0.8086 - val_loss: 0.4402 - val_acc: 0.8266


In [17]:
model.evaluate(input_test, y_test)

25000/25000 [==============================] - 74s 3ms/step


[0.4345056212997436, 0.82468]